In [10]:
import importlib
import sys
import time
 
importlib.reload(sys)
time1 = time.time()
# print("初始时间为：",time1)
 
import os.path

# pdfminer 库
from pdfminer.pdfparser import  PDFParser,PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed

In [28]:
def parse(pdf_path, output_file):
    '''解析PDF文本，并保存到TXT文件中'''
    fp = open(pdf_path,'rb')
    #用文件对象创建一个PDF文档分析器
    parser = PDFParser(fp)
    #创建一个PDF文档
    doc = PDFDocument()
    #连接分析器，与文档对象
    parser.set_document(doc)
    doc.set_parser(parser)
 
    #提供初始化密码，如果没有密码，就创建一个空的字符串
    doc.initialize()
 
    #检测文档是否提供txt转换，不提供就忽略
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        #创建PDF，资源管理器，来共享资源
        rsrcmgr = PDFResourceManager()
        #创建一个PDF设备对象
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr,laparams=laparams)
        #创建一个PDF解释其对象
        interpreter = PDFPageInterpreter(rsrcmgr,device)
 
        #循环遍历列表，每次处理一个page内容
        # doc.get_pages() 获取page列表
        
        if not os.path.isfile(output_file):  # 无文件时创建
            fd = open(output_file, mode="w")
            fd.close()
        else:
            fd = open(output_file, mode="w")
            fd.truncate()
            pass

        # 作为演示，我们只打印首页前3行
        LIMIT_PAGES = 1
        LIMIT_ROWS = 3
        cur_page = 0
        for page in doc.get_pages():
            if cur_page>=LIMIT_PAGES:
                break
            cur_page += 1
            interpreter.process_page(page)
            #接受该页面的LTPage对象
            layout = device.get_result()
            # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象
            # 一般包括LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal 等等
            # 想要获取文本就获得对象的text属性，
            cur_row = 0
            for x in layout:
                if cur_row>=LIMIT_ROWS:
                    break
                cur_row += 1
                if(isinstance(x,LTTextBoxHorizontal)):
                    with open(output_file,'a') as f:
                        results = x.get_text()
                        print(results)
                        f.write(results  +"\n")

In [29]:
!ls

Day6.ipynb ReadMe.txt result.txt test.pdf


In [34]:
if __name__ == '__main__':
    pdf_path = 'test.pdf'
    output_file = 'result.txt'
    parse(pdf_path, output_file)
    time2 = time.time()
    print("总共消耗时间为:{time}毫秒".format(time = time2-time1))

1. PDFMiner是⼀一个从PDF⽂文档中提取信息的⼯工具。与其他pdf相关的
⼯工具不不同，它完全专注于获取和分析⽂文本数据。PDFMiner允许获取
⻚页⾯面中⽂文本的确切位置，以及其他信息，⽐比如字体或⾏行行。它包括⼀一
个PDF转换器器，可以将PDF⽂文件转换成其他⽂文本格式(如HTML)。
它有⼀一个可扩展的PDF解析器器，可以⽤用于其他⽬目的⽽而不不是⽂文本分析。

2. Python3安装pdfminer的⽅方式：pip install pdfminer3k

3.pdfminer对⽂文本解析优化，对图⽚片解析不不那么友好。

总共消耗时间为:700.1334829330444毫秒
